In [263]:
import pandas as pd
import numpy as np
import seaborn as sns
!pip install pyarrow
import pyarrow
sns.set()
import gc
gc.collect()
import matplotlib.pyplot as plt  
# plt.style.use('seaborn-notebook')
# COLOR = 'white'
# plt.rcParams['text.color'] = COLOR
# plt.rcParams['axes.labelcolor'] = COLOR
# plt.rcParams['xtick.color'] = COLOR
# plt.rcParams['ytick.color'] = COLOR
%matplotlib inline
from tqdm import tqdm
tqdm.pandas()

from pyhive import presto
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")
import math
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [264]:
conn = presto.connect(
    host="bi-presto.serving.data.production.internal",
    #host="presto.processing.yoda.run",
    #host="prime-trino.serving.data.production.internal",
    port=80,
    protocol="http",
    catalog="hive",
    username="naveen.verma@rapido.bike",
)


def run_query(query):
    return pd.read_sql_query(query, conn)

In [265]:
d_query = f"""
 select captain_id,order_id,service_obj_service_name, updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude ,order_status,event_type
from orders.order_logs_immutable
where  yyyymmdd between '20230306' and '20230306'
    and order_status in ('dropped','onTheWay')
    and event_type in ('dropped','accepted')
    and service_obj_service_name in ('Link','Auto')
order by captain_id,updated_epoch desc
    """
data = run_query(d_query)
data

KeyboardInterrupt: 

In [ ]:
data.to_csv('loc1.csv',index=False)

In [266]:
data =  pd.read_csv('loc1.csv') 
data

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type
0,5737c6c2ddbec2203f733224,6405ed161d8aa83196be2c35,Auto,1678110154861,20230306,12.967134,77.657667,onTheWay,accepted
1,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678109375353,20230306,12.964447,77.657547,dropped,dropped
2,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678108396402,20230306,12.977097,77.674790,onTheWay,accepted
3,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678107646305,20230306,12.976640,77.671814,dropped,dropped
4,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678106515796,20230306,12.964131,77.649366,onTheWay,accepted
...,...,...,...,...,...,...,...,...,...
1564607,62b8550617af9a7d3762d0ec,64058b572030b56c339c0039,Auto,1678084982854,20230306,18.609151,73.751108,onTheWay,accepted
1564608,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678080717357,20230306,18.612537,73.749237,dropped,dropped
1564609,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678079374496,20230306,18.671108,73.731655,onTheWay,accepted
1564610,62b8550617af9a7d3762d0ec,640574d44f487150d78dad5b,Auto,1678079200729,20230306,18.671096,73.731640,onTheWay,accepted


In [267]:
def convert_to_datetime(updated_epoch):
         
 base_datetime = datetime(1970, 1, 1,5,30)
 delta = timedelta(0, 0, 0, float(updated_epoch))
 return base_datetime + delta

data['createdon'] = data[['updated_epoch']].apply(lambda x: convert_to_datetime(x[0]), axis=1)
data

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon
0,5737c6c2ddbec2203f733224,6405ed161d8aa83196be2c35,Auto,1678110154861,20230306,12.967134,77.657667,onTheWay,accepted,2023-03-06 19:12:34.861
1,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678109375353,20230306,12.964447,77.657547,dropped,dropped,2023-03-06 18:59:35.353
2,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678108396402,20230306,12.977097,77.674790,onTheWay,accepted,2023-03-06 18:43:16.402
3,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678107646305,20230306,12.976640,77.671814,dropped,dropped,2023-03-06 18:30:46.305
4,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678106515796,20230306,12.964131,77.649366,onTheWay,accepted,2023-03-06 18:11:55.796
...,...,...,...,...,...,...,...,...,...,...
1564607,62b8550617af9a7d3762d0ec,64058b572030b56c339c0039,Auto,1678084982854,20230306,18.609151,73.751108,onTheWay,accepted,2023-03-06 12:13:02.854
1564608,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678080717357,20230306,18.612537,73.749237,dropped,dropped,2023-03-06 11:01:57.357
1564609,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678079374496,20230306,18.671108,73.731655,onTheWay,accepted,2023-03-06 10:39:34.496
1564610,62b8550617af9a7d3762d0ec,640574d44f487150d78dad5b,Auto,1678079200729,20230306,18.671096,73.731640,onTheWay,accepted,2023-03-06 10:36:40.729


In [268]:
data['lag_values_long'] = data.sort_values(by = ['updated_epoch']).groupby(['captain_id','yyyymmdd'])['captain_location_longitude'].shift(1)
data['lag_values_lat'] = data.sort_values(by = ['updated_epoch']).groupby(['captain_id','yyyymmdd'])['captain_location_latitude'].shift(1)
data['lag_epoch'] = data.sort_values(by = ['updated_epoch']).groupby(['captain_id','yyyymmdd'])['createdon'].shift(1)
data

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch
0,5737c6c2ddbec2203f733224,6405ed161d8aa83196be2c35,Auto,1678110154861,20230306,12.967134,77.657667,onTheWay,accepted,2023-03-06 19:12:34.861,77.657547,12.964447,2023-03-06 18:59:35.353
1,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678109375353,20230306,12.964447,77.657547,dropped,dropped,2023-03-06 18:59:35.353,77.674790,12.977097,2023-03-06 18:43:16.402
2,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678108396402,20230306,12.977097,77.674790,onTheWay,accepted,2023-03-06 18:43:16.402,77.671814,12.976640,2023-03-06 18:30:46.305
3,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678107646305,20230306,12.976640,77.671814,dropped,dropped,2023-03-06 18:30:46.305,77.649366,12.964131,2023-03-06 18:11:55.796
4,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678106515796,20230306,12.964131,77.649366,onTheWay,accepted,2023-03-06 18:11:55.796,77.649418,12.964144,2023-03-06 17:41:50.382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564607,62b8550617af9a7d3762d0ec,64058b572030b56c339c0039,Auto,1678084982854,20230306,18.609151,73.751108,onTheWay,accepted,2023-03-06 12:13:02.854,73.749237,18.612537,2023-03-06 11:01:57.357
1564608,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678080717357,20230306,18.612537,73.749237,dropped,dropped,2023-03-06 11:01:57.357,73.731655,18.671108,2023-03-06 10:39:34.496
1564609,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678079374496,20230306,18.671108,73.731655,onTheWay,accepted,2023-03-06 10:39:34.496,73.731640,18.671096,2023-03-06 10:36:40.729
1564610,62b8550617af9a7d3762d0ec,640574d44f487150d78dad5b,Auto,1678079200729,20230306,18.671096,73.731640,onTheWay,accepted,2023-03-06 10:36:40.729,73.756760,18.592413,2023-03-06 09:54:40.510


In [269]:
from math import sin, cos, sqrt, atan2,radians,asin


def haversine(captain_location_longitude, captain_location_latitude, lag_values_long, lag_values_lat):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    captain_location_longitude, captain_location_latitude, lag_values_long, lag_values_lat = map(radians, [captain_location_longitude, captain_location_latitude, lag_values_long, lag_values_lat])
    # haversine formula 
    dlon = lag_values_long - captain_location_longitude 
    dlat = lag_values_lat - captain_location_latitude 
    a = sin(dlat/2)**2 + cos(captain_location_latitude) * cos(lag_values_lat) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

data['distance'] = data.apply(lambda x:haversine(x['captain_location_latitude'], x['captain_location_longitude'], x['lag_values_lat'], x['lag_values_long']) * 0.90, axis = 1)

data

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance
0,5737c6c2ddbec2203f733224,6405ed161d8aa83196be2c35,Auto,1678110154861,20230306,12.967134,77.657667,onTheWay,accepted,2023-03-06 19:12:34.861,77.657547,12.964447,2023-03-06 18:59:35.353,0.058724
1,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678109375353,20230306,12.964447,77.657547,dropped,dropped,2023-03-06 18:59:35.353,77.674790,12.977097,2023-03-06 18:43:16.402,1.746692
2,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678108396402,20230306,12.977097,77.674790,onTheWay,accepted,2023-03-06 18:43:16.402,77.671814,12.976640,2023-03-06 18:30:46.305,0.298019
3,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678107646305,20230306,12.976640,77.671814,dropped,dropped,2023-03-06 18:30:46.305,77.649366,12.964131,2023-03-06 18:11:55.796,2.262371
4,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678106515796,20230306,12.964131,77.649366,onTheWay,accepted,2023-03-06 18:11:55.796,77.649418,12.964144,2023-03-06 17:41:50.382,0.005220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564607,62b8550617af9a7d3762d0ec,64058b572030b56c339c0039,Auto,1678084982854,20230306,18.609151,73.751108,onTheWay,accepted,2023-03-06 12:13:02.854,73.749237,18.612537,2023-03-06 11:01:57.357,0.209911
1564608,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678080717357,20230306,18.612537,73.749237,dropped,dropped,2023-03-06 11:01:57.357,73.731655,18.671108,2023-03-06 10:39:34.496,2.406115
1564609,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678079374496,20230306,18.671108,73.731655,onTheWay,accepted,2023-03-06 10:39:34.496,73.731640,18.671096,2023-03-06 10:36:40.729,0.001460
1564610,62b8550617af9a7d3762d0ec,640574d44f487150d78dad5b,Auto,1678079200729,20230306,18.671096,73.731640,onTheWay,accepted,2023-03-06 10:36:40.729,73.756760,18.592413,2023-03-06 09:54:40.510,3.343308


In [270]:
data['diff_time'] = data['createdon']  - data['lag_epoch']
data

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance,diff_time
0,5737c6c2ddbec2203f733224,6405ed161d8aa83196be2c35,Auto,1678110154861,20230306,12.967134,77.657667,onTheWay,accepted,2023-03-06 19:12:34.861,77.657547,12.964447,2023-03-06 18:59:35.353,0.058724,0 days 00:12:59.508000
1,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678109375353,20230306,12.964447,77.657547,dropped,dropped,2023-03-06 18:59:35.353,77.674790,12.977097,2023-03-06 18:43:16.402,1.746692,0 days 00:16:18.951000
2,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678108396402,20230306,12.977097,77.674790,onTheWay,accepted,2023-03-06 18:43:16.402,77.671814,12.976640,2023-03-06 18:30:46.305,0.298019,0 days 00:12:30.097000
3,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678107646305,20230306,12.976640,77.671814,dropped,dropped,2023-03-06 18:30:46.305,77.649366,12.964131,2023-03-06 18:11:55.796,2.262371,0 days 00:18:50.509000
4,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678106515796,20230306,12.964131,77.649366,onTheWay,accepted,2023-03-06 18:11:55.796,77.649418,12.964144,2023-03-06 17:41:50.382,0.005220,0 days 00:30:05.414000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564607,62b8550617af9a7d3762d0ec,64058b572030b56c339c0039,Auto,1678084982854,20230306,18.609151,73.751108,onTheWay,accepted,2023-03-06 12:13:02.854,73.749237,18.612537,2023-03-06 11:01:57.357,0.209911,0 days 01:11:05.497000
1564608,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678080717357,20230306,18.612537,73.749237,dropped,dropped,2023-03-06 11:01:57.357,73.731655,18.671108,2023-03-06 10:39:34.496,2.406115,0 days 00:22:22.861000
1564609,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678079374496,20230306,18.671108,73.731655,onTheWay,accepted,2023-03-06 10:39:34.496,73.731640,18.671096,2023-03-06 10:36:40.729,0.001460,0 days 00:02:53.767000
1564610,62b8550617af9a7d3762d0ec,640574d44f487150d78dad5b,Auto,1678079200729,20230306,18.671096,73.731640,onTheWay,accepted,2023-03-06 10:36:40.729,73.756760,18.592413,2023-03-06 09:54:40.510,3.343308,0 days 00:42:00.219000


In [271]:
data['diff_time'] = data['diff_time'].astype('timedelta64[m]')
data

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance,diff_time
0,5737c6c2ddbec2203f733224,6405ed161d8aa83196be2c35,Auto,1678110154861,20230306,12.967134,77.657667,onTheWay,accepted,2023-03-06 19:12:34.861,77.657547,12.964447,2023-03-06 18:59:35.353,0.058724,12.0
1,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678109375353,20230306,12.964447,77.657547,dropped,dropped,2023-03-06 18:59:35.353,77.674790,12.977097,2023-03-06 18:43:16.402,1.746692,16.0
2,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678108396402,20230306,12.977097,77.674790,onTheWay,accepted,2023-03-06 18:43:16.402,77.671814,12.976640,2023-03-06 18:30:46.305,0.298019,12.0
3,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678107646305,20230306,12.976640,77.671814,dropped,dropped,2023-03-06 18:30:46.305,77.649366,12.964131,2023-03-06 18:11:55.796,2.262371,18.0
4,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678106515796,20230306,12.964131,77.649366,onTheWay,accepted,2023-03-06 18:11:55.796,77.649418,12.964144,2023-03-06 17:41:50.382,0.005220,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564607,62b8550617af9a7d3762d0ec,64058b572030b56c339c0039,Auto,1678084982854,20230306,18.609151,73.751108,onTheWay,accepted,2023-03-06 12:13:02.854,73.749237,18.612537,2023-03-06 11:01:57.357,0.209911,71.0
1564608,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678080717357,20230306,18.612537,73.749237,dropped,dropped,2023-03-06 11:01:57.357,73.731655,18.671108,2023-03-06 10:39:34.496,2.406115,22.0
1564609,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678079374496,20230306,18.671108,73.731655,onTheWay,accepted,2023-03-06 10:39:34.496,73.731640,18.671096,2023-03-06 10:36:40.729,0.001460,2.0
1564610,62b8550617af9a7d3762d0ec,640574d44f487150d78dad5b,Auto,1678079200729,20230306,18.671096,73.731640,onTheWay,accepted,2023-03-06 10:36:40.729,73.756760,18.592413,2023-03-06 09:54:40.510,3.343308,42.0


In [272]:
data['speed'] = data['distance']*60/data['diff_time']
data

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance,diff_time,speed
0,5737c6c2ddbec2203f733224,6405ed161d8aa83196be2c35,Auto,1678110154861,20230306,12.967134,77.657667,onTheWay,accepted,2023-03-06 19:12:34.861,77.657547,12.964447,2023-03-06 18:59:35.353,0.058724,12.0,0.293619
1,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678109375353,20230306,12.964447,77.657547,dropped,dropped,2023-03-06 18:59:35.353,77.674790,12.977097,2023-03-06 18:43:16.402,1.746692,16.0,6.550095
2,5737c6c2ddbec2203f733224,6405e6e38183376d1871c28d,Auto,1678108396402,20230306,12.977097,77.674790,onTheWay,accepted,2023-03-06 18:43:16.402,77.671814,12.976640,2023-03-06 18:30:46.305,0.298019,12.0,1.490093
3,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678107646305,20230306,12.976640,77.671814,dropped,dropped,2023-03-06 18:30:46.305,77.649366,12.964131,2023-03-06 18:11:55.796,2.262371,18.0,7.541236
4,5737c6c2ddbec2203f733224,6405df078183376d1871b3c3,Auto,1678106515796,20230306,12.964131,77.649366,onTheWay,accepted,2023-03-06 18:11:55.796,77.649418,12.964144,2023-03-06 17:41:50.382,0.005220,30.0,0.010441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564607,62b8550617af9a7d3762d0ec,64058b572030b56c339c0039,Auto,1678084982854,20230306,18.609151,73.751108,onTheWay,accepted,2023-03-06 12:13:02.854,73.749237,18.612537,2023-03-06 11:01:57.357,0.209911,71.0,0.177390
1564608,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678080717357,20230306,18.612537,73.749237,dropped,dropped,2023-03-06 11:01:57.357,73.731655,18.671108,2023-03-06 10:39:34.496,2.406115,22.0,6.562131
1564609,62b8550617af9a7d3762d0ec,640575879479fa4c8cfc04d3,Auto,1678079374496,20230306,18.671108,73.731655,onTheWay,accepted,2023-03-06 10:39:34.496,73.731640,18.671096,2023-03-06 10:36:40.729,0.001460,2.0,0.043791
1564610,62b8550617af9a7d3762d0ec,640574d44f487150d78dad5b,Auto,1678079200729,20230306,18.671096,73.731640,onTheWay,accepted,2023-03-06 10:36:40.729,73.756760,18.592413,2023-03-06 09:54:40.510,3.343308,42.0,4.776154


In [273]:
data['flag'] = np.where((data['speed']>200) & (data['distance']>10) ,1,0)

#data1.replace([np.inf, -np.inf], np.nan, inplace=True)
data1 = data[data.flag ==1]
data1 = data1[data1.diff_time!=0]
data1

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance,diff_time,speed,flag
117873,5c7b9bd4875c9853625f43b3,6405dee763d00d05e3a79499,Link,1678107275970,20230306,28.361942,76.939987,dropped,dropped,2023-03-06 18:24:35.970,77.339485,28.561922,2023-03-06 18:23:22.668,40.227233,1.0,2413.633985,1
275484,5d50e30555fbf50d45f00c66,640549d02030b56c339b7be8,Link,1678069050887,20230306,17.407391,78.496976,onTheWay,accepted,2023-03-06 07:47:30.887,78.378331,17.353343,2023-03-06 07:43:54.355,11.922879,3.0,238.457590,1
1337114,622d88abbbd0607a05e85816,6404e7db275d6f525c9467bd,Link,1678043403062,20230306,28.661657,77.226616,dropped,dropped,2023-03-06 00:40:03.062,77.116336,28.421334,2023-03-06 00:36:52.922,12.260337,3.0,245.206735,1


In [274]:
flagged =  pd.DataFrame(data1['captain_id'].drop_duplicates())

flagged

,captain_id
117873,5c7b9bd4875c9853625f43b3
275484,5d50e30555fbf50d45f00c66
1337114,622d88abbbd0607a05e85816


In [275]:
data2 = data.merge(flagged,on='captain_id',how='right')
data2

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance,diff_time,speed,flag
0,5c7b9bd4875c9853625f43b3,6405dee763d00d05e3a79499,Link,1678107275970,20230306,28.361942,76.939987,dropped,dropped,2023-03-06 18:24:35.970,77.339485,28.561922,2023-03-06 18:23:22.668,40.227233,1.0,2413.633985,1
1,5c7b9bd4875c9853625f43b3,6405e2071cfa597c3857c7dc,Link,1678107202668,20230306,28.561922,77.339485,onTheWay,accepted,2023-03-06 18:23:22.668,76.940268,28.385248,2023-03-06 18:09:38.983,40.145124,13.0,185.285190,0
2,5c7b9bd4875c9853625f43b3,6405dee763d00d05e3a79499,Link,1678106378983,20230306,28.385248,76.940268,onTheWay,accepted,2023-03-06 18:09:38.983,77.032600,28.445225,2023-03-06 08:22:11.904,9.338550,587.0,0.954537,0
3,5c7b9bd4875c9853625f43b3,64054f261d8aa83196bd118d,Link,1678071131904,20230306,28.445225,77.032600,dropped,dropped,2023-03-06 08:22:11.904,77.014069,28.490686,2023-03-06 07:55:55.202,2.117311,26.0,4.886103,0
4,5c7b9bd4875c9853625f43b3,64054f261d8aa83196bd118d,Link,1678069555202,20230306,28.490686,77.014069,onTheWay,accepted,2023-03-06 07:55:55.202,NaN,NaN,NaT,NaN,NaN,NaN,0
5,5d50e30555fbf50d45f00c66,6405dde0275d6f525c95cb3f,Link,1678106101282,20230306,17.368229,78.390464,onTheWay,accepted,2023-03-06 18:05:01.282,78.390528,17.368220,2023-03-06 18:00:38.943,0.006328,4.0,0.094914,0
6,5d50e30555fbf50d45f00c66,6405dce66e12e25c673a5dd6,Link,1678105838943,20230306,17.368220,78.390528,onTheWay,accepted,2023-03-06 18:00:38.943,78.352461,17.390265,2023-03-06 12:07:17.598,3.835448,353.0,0.651918,0
7,5d50e30555fbf50d45f00c66,640589f101b8b86aa8bcbac6,Link,1678084637598,20230306,17.390265,78.352461,onTheWay,accepted,2023-03-06 12:07:17.598,78.353159,17.494367,2023-03-06 11:01:58.640,2.104407,65.0,1.942530,0
8,5d50e30555fbf50d45f00c66,64057ab77ce00b281750ca73,Link,1678080718640,20230306,17.494367,78.353159,onTheWay,accepted,2023-03-06 11:01:58.640,78.356440,17.495325,2023-03-06 11:00:54.123,0.328937,1.0,19.736214,0
9,5d50e30555fbf50d45f00c66,64057a74275d6f525c95284e,Link,1678080654123,20230306,17.495325,78.356440,onTheWay,accepted,2023-03-06 11:00:54.123,78.358499,17.515858,2023-03-06 10:44:13.890,0.463055,16.0,1.736455,0


In [276]:
data2['captain_id'].unique()

array(['5c7b9bd4875c9853625f43b3', '5d50e30555fbf50d45f00c66',
       '622d88abbbd0607a05e85816'], dtype=object)

In [277]:
data2 = data2.sort_values(by=['captain_id']).sort_values(by = ['updated_epoch'],ascending = False)
data2

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance,diff_time,speed,flag
0,5c7b9bd4875c9853625f43b3,6405dee763d00d05e3a79499,Link,1678107275970,20230306,28.361942,76.939987,dropped,dropped,2023-03-06 18:24:35.970,77.339485,28.561922,2023-03-06 18:23:22.668,40.227233,1.0,2413.633985,1
1,5c7b9bd4875c9853625f43b3,6405e2071cfa597c3857c7dc,Link,1678107202668,20230306,28.561922,77.339485,onTheWay,accepted,2023-03-06 18:23:22.668,76.940268,28.385248,2023-03-06 18:09:38.983,40.145124,13.0,185.285190,0
2,5c7b9bd4875c9853625f43b3,6405dee763d00d05e3a79499,Link,1678106378983,20230306,28.385248,76.940268,onTheWay,accepted,2023-03-06 18:09:38.983,77.032600,28.445225,2023-03-06 08:22:11.904,9.338550,587.0,0.954537,0
5,5d50e30555fbf50d45f00c66,6405dde0275d6f525c95cb3f,Link,1678106101282,20230306,17.368229,78.390464,onTheWay,accepted,2023-03-06 18:05:01.282,78.390528,17.368220,2023-03-06 18:00:38.943,0.006328,4.0,0.094914,0
6,5d50e30555fbf50d45f00c66,6405dce66e12e25c673a5dd6,Link,1678105838943,20230306,17.368220,78.390528,onTheWay,accepted,2023-03-06 18:00:38.943,78.352461,17.390265,2023-03-06 12:07:17.598,3.835448,353.0,0.651918,0
7,5d50e30555fbf50d45f00c66,640589f101b8b86aa8bcbac6,Link,1678084637598,20230306,17.390265,78.352461,onTheWay,accepted,2023-03-06 12:07:17.598,78.353159,17.494367,2023-03-06 11:01:58.640,2.104407,65.0,1.942530,0
8,5d50e30555fbf50d45f00c66,64057ab77ce00b281750ca73,Link,1678080718640,20230306,17.494367,78.353159,onTheWay,accepted,2023-03-06 11:01:58.640,78.356440,17.495325,2023-03-06 11:00:54.123,0.328937,1.0,19.736214,0
9,5d50e30555fbf50d45f00c66,64057a74275d6f525c95284e,Link,1678080654123,20230306,17.495325,78.356440,onTheWay,accepted,2023-03-06 11:00:54.123,78.358499,17.515858,2023-03-06 10:44:13.890,0.463055,16.0,1.736455,0
10,5d50e30555fbf50d45f00c66,6405763601b8b86aa8bc9a78,Link,1678079653890,20230306,17.515858,78.358499,onTheWay,accepted,2023-03-06 10:44:13.890,78.361719,17.510189,2023-03-06 10:41:52.227,0.341976,2.0,10.259278,0
11,5d50e30555fbf50d45f00c66,6405759e4ad65d29deec00bb,Link,1678079512227,20230306,17.510189,78.361719,onTheWay,accepted,2023-03-06 10:41:52.227,78.362238,17.506883,2023-03-06 10:36:55.021,0.084582,4.0,1.268737,0


In [279]:
data = data.sort_values(by = ['updated_epoch'],ascending= False)
sample = data[data['captain_id']=='5c7b9bd4875c9853625f43b3']
sample = sample.sort_values(by = ['updated_epoch'],ascending= False)
sample 

,captain_id,order_id,service_obj_service_name,updated_epoch,yyyymmdd,captain_location_latitude,captain_location_longitude,order_status,event_type,createdon,lag_values_long,lag_values_lat,lag_epoch,distance,diff_time,speed,flag
117873,5c7b9bd4875c9853625f43b3,6405dee763d00d05e3a79499,Link,1678107275970,20230306,28.361942,76.939987,dropped,dropped,2023-03-06 18:24:35.970,77.339485,28.561922,2023-03-06 18:23:22.668,40.227233,1.0,2413.633985,1
117874,5c7b9bd4875c9853625f43b3,6405e2071cfa597c3857c7dc,Link,1678107202668,20230306,28.561922,77.339485,onTheWay,accepted,2023-03-06 18:23:22.668,76.940268,28.385248,2023-03-06 18:09:38.983,40.145124,13.0,185.285190,0
117875,5c7b9bd4875c9853625f43b3,6405dee763d00d05e3a79499,Link,1678106378983,20230306,28.385248,76.940268,onTheWay,accepted,2023-03-06 18:09:38.983,77.032600,28.445225,2023-03-06 08:22:11.904,9.338550,587.0,0.954537,0
117876,5c7b9bd4875c9853625f43b3,64054f261d8aa83196bd118d,Link,1678071131904,20230306,28.445225,77.032600,dropped,dropped,2023-03-06 08:22:11.904,77.014069,28.490686,2023-03-06 07:55:55.202,2.117311,26.0,4.886103,0
117877,5c7b9bd4875c9853625f43b3,64054f261d8aa83196bd118d,Link,1678069555202,20230306,28.490686,77.014069,onTheWay,accepted,2023-03-06 07:55:55.202,NaN,NaN,NaT,NaN,NaN,NaN,0
